In [68]:
import pandas as pd 
import json
import math

In [69]:
file = './Data/HateXplain.json'
with open(file) as train_file:
    dict_train = json.load(train_file)

In [70]:
train = pd.DataFrame.from_dict(dict_train, orient='index')
train.reset_index(level=0, inplace=True)

In [71]:
train

,index,post_id,annotators,rationales,post_tokens
0,1179055004553900032_twitter,1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[i, dont, think, im, getting, my, baby, them, ..."
1,1179063826874032128_twitter,1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[we, cannot, continue, calling, ourselves, fem..."
2,1178793830532956161_twitter,1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],"[nawt, yall, niggers, ignoring, me]"
3,1179088797964763136_twitter,1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[<user>, i, am, bit, confused, coz, chinese, p..."
4,1179085312976445440_twitter,1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[this, bitch, in, whataburger, eating, a, burg..."
...,...,...,...,...,...
20143,9989999_gab,9989999_gab,"[{'label': 'offensive', 'annotator_id': 217, '...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,...","[if, ur, still, on, twitter, tell, carlton, i,..."
20144,9990225_gab,9990225_gab,"[{'label': 'offensive', 'annotator_id': 220, '...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...","[when, i, first, got, on, here, and, said, i, ..."
20145,9991681_gab,9991681_gab,"[{'label': 'offensive', 'annotator_id': 206, '...",[],"[was, macht, der, moslem, wenn, der, zion, geg..."
20146,9992513_gab,9992513_gab,"[{'label': 'hatespeech', 'annotator_id': 209, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...","[it, is, awful, look, at, world, demographics,..."


In [72]:
def annotatorSelect(i):

    annontators = train["annotators"][i]
    annotator = annontators[math.floor(random.uniform(0, 1)*len(annontators))]
    annotator

    return annotator


In [75]:
for i in range(len(train)):
    print(annotatorSelect(i))

{'label': 'normal', 'annotator_id': 3, 'target': ['None']}
{'label': 'normal', 'annotator_id': 3, 'target': ['None']}
{'label': 'hatespeech', 'annotator_id': 3, 'target': ['African']}
{'label': 'offensive', 'annotator_id': 4, 'target': ['Asian']}
{'label': 'hatespeech', 'annotator_id': 2, 'target': ['Women', 'Caucasian']}
{'label': 'hatespeech', 'annotator_id': 1, 'target': ['Jewish']}
{'label': 'hatespeech', 'annotator_id': 3, 'target': ['African']}
{'label': 'offensive', 'annotator_id': 5, 'target': ['Jewish']}
{'label': 'hatespeech', 'annotator_id': 4, 'target': ['African', 'Jewish']}
{'label': 'hatespeech', 'annotator_id': 3, 'target': ['African']}
{'label': 'hatespeech', 'annotator_id': 3, 'target': ['Islam']}
{'label': 'normal', 'annotator_id': 4, 'target': ['African']}
{'label': 'normal', 'annotator_id': 8, 'target': ['None']}
{'label': 'normal', 'annotator_id': 6, 'target': ['None']}
{'label': 'offensive', 'annotator_id': 8, 'target': ['Women']}
{'label': 'normal', 'annotator_i